# Import

In [175]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
import cv2
from subprocess import check_output
import os
import csv

# Parameters

In [176]:
labelsPath = 'C:/Users/ANTHO/Desktop/TestTensorFlow/KAGGLE/Dataset/train.csv'

epochs=2000
batchSize=100
learningRate=1e-2
#dropOutFactor=0.5
trainSize=700
testSize=200


# Extract the Labels

#Matrix with all the labels and 
with open(labelsPath, 'r') as f:
    reader = csv.reader(f)
    labels = np.matrix(list(reader), dtype=float)/100

In [177]:
with open(labelsPath, 'r') as f:
    reader = csv.reader(f)
    labels = np.matrix(list(reader))
    labels = labels[1:]#Remove the header line

    
labelsIn=labels[...,6]
labelsOut=labels[...,1:6]
    

# Select size of train and test 

In [178]:
#Size of the training set
sizeTot= (labels.shape[0]) #Nb of image in the dataset (from 0)

if (sizeTot-trainSize-testSize)<0:
    testSize=sizeTot-trainSize
    print('Not enough data for the test set')
    print('Only ', testSize, ' used for the test dataset')

labelsInTest = labelsIn[(sizeTot-testSize):]
labelsOutTest = labelsOut[(sizeTot-testSize):]

if trainSize>sizeTot:
    trainSize=sizeTot
    print('Size of the training data bigger than the dataset !')
#Training set
labelsInTrain=labelsIn[0:trainSize]
labelsOutTrain=labelsOut[0:trainSize]


# Convolutional Neural Network

In [179]:
#Function
def compute_accuracy(v_xs, v_ys):
    global prediction
    y_pre = sess.run(prediction, feed_dict={xs: v_xs})
    y_pre = tf.cast(y_pre, tf.float32)#Force float32
    #correct_prediction = tf.equal(tf.argmax(y_pre,1), tf.argmax(v_ys,1))
    #accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    accuracy = tf.reduce_sum(tf.square((tf.subtract(y_pre,  v_ys))))
    result = sess.run(accuracy, feed_dict={xs: v_xs, ys: v_ys})
    return result

def weight_variable(shape):
    initial = tf.truncated_normal(shape=shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)


In [180]:
## Architecture

## Input layer ##
with tf.name_scope("Input") as scope:
    # define placeholder for inputs to network
    #input
    xs = tf.placeholder(tf.float32, [None, 1])
    #output
    ys = tf.placeholder(tf.float32, [None, 5])
    

## fc1 layer ##
with tf.name_scope("FC1") as scope:
    sizeIn=1
    sizeOut=10
    W_fc1 = weight_variable([sizeIn, sizeOut])
    b_fc1 = bias_variable([sizeOut])
    h_fc1 = tf.nn.relu(tf.matmul(xs, W_fc1) + b_fc1)

## fc2 layer ##
with tf.name_scope("FC2") as scope:
    sizeIn=sizeOut
    sizeOut=5
    W_fc2 = weight_variable([sizeIn, sizeOut])
    b_fc2 = bias_variable([sizeOut])
    prediction = tf.matmul(h_fc1, W_fc2) + b_fc2

## Loss function ##
# the error between prediction and real data
#cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(prediction),reduction_indices=[1]))# loss
with tf.name_scope("cost_function") as scope:
    cross_entropy = tf.reduce_sum(tf.square((tf.subtract(prediction, ys))))#Euclidean distance
    tf.summary.scalar("cost_function", cross_entropy)
    
with tf.name_scope("train") as scope:
    train_step = tf.train.AdamOptimizer(learningRate).minimize(cross_entropy)
    #train_step = tf.train.GradientDescentOptimizer(0.9).minimize(cross_entropy)
    


In [181]:
#Training
if int((tf.__version__).split('.')[1]) < 12 and int((tf.__version__).split('.')[0]) < 1:
    init = tf.initialize_all_variables()
else:
    init = tf.global_variables_initializer()
# Merge all summaries into a single operator
merged_summary_op = tf.summary.merge_all()

saver = tf.train.Saver([W_fc1,b_fc1, W_fc2, b_fc2])

with tf.Session() as sess:
#sess = tf.Session()
# important step
# tf.initialize_all_variables() no long valid from
# 2017-03-02 if using tensorflow >= 0.12

    sess.run(init)
    
    # Folder where the data are saved
    summary_writer = tf.summary.FileWriter('C:/Users/ANTHO/Desktop/TestTensorFlow/KAGGLE/Measure/Classes/', sess.graph)

    if int(len(labelsInTrain)) < batchSize:#If there are less data than the batch size
        nbBatch=1
    else:
        nbBatch=int((len(labelsInTrain))/batchSize)

    #print(compute_accuracy(imgsTest, labelsHotTest))
    for i in range(epochs):#Go through all the epochs
        for batchNum in range(nbBatch):#Go through all the batches
            if batchSize==1:
                batch_xs = labelsInTrain[batchNum*batchSize]
                batch_ys = labelsOutTrain[batchNum*batchSize]
            else:
                batch_xs = labelsInTrain[batchNum*batchSize:(batchNum*batchSize+(batchSize-1))]
                batch_ys = labelsOutTrain[batchNum*batchSize:(batchNum*batchSize+(batchSize-1))]

            sess.run(train_step, feed_dict={xs: batch_xs, ys: batch_ys})
        if i % 100 == 0:
            print(i)
            print(compute_accuracy(labelsInTest, labelsOutTest))
            saver.save(sess, './Measure/my-model', global_step=i)
            
            #summary_str = sess.run(merged_summary_op, feed_dict={xs: batch_xs, ys: batch_ys, keep_prob: dropOutFactor})
            #summary_writer.add_summary(summary_str, i*int(len(imgs)/batchSize) + batchNum)
            #cost_history = np.append(cost_history,sess.run(cross_entropy,feed_dict={xs: batch_xs, ys: batch_ys, keep_prob: dropOutFactor}))
            #weigthMeasure = sess.run(W_conv1,feed_dict={xs: batch_xs, ys: batch_ys, keep_prob: dropOutFactor})
        #if i % 1 == 0:
            #print(i)
            #print(weigthMeasure)
            #print(compute_accuracy(imgsTest, labelsHotTest)) 



0
2.39981e+06
100
367480.0
200
351830.0
300
348746.0
400
349236.0
500
347464.0
600
347454.0
700
346910.0
800
346756.0
900
347319.0
1000
346288.0
1100
346679.0
1200
345789.0
1300
346323.0
1400
346342.0
1500
346047.0
1600
347027.0
1700
346457.0
1800
347066.0
1900
345993.0
